
              © 2025 Nokia
              Licensed under the BSD 3-Clause Clear License
              SPDX-License-Identifier: BSD-3-Clause-Clear

In [ ]:
!pip install seaborn

In [ ]:
import time
import pandas as pd
from io import StringIO
import seaborn as sns 
import matplotlib.pyplot as plt

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for this experiment

In [ ]:
# configure: host name, nic, cores, ram, disk according to your needs. 


slice_name="wan-same-site-3-nodes-same-host-v2" + fablib.get_bastion_username()

node_conf = [
 {'name': "sender",    'host': 'newy-w2.fabric-testbed.net', 'nic': 'NIC_ConnectX_5', 'cores': 32, 'ram': 96, 'disk': 40, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "receiver",  'host': 'newy-w2.fabric-testbed.net', 'nic': 'NIC_Basic', 'cores': 32, 'ram': 96, 'disk': 40, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}, 
 {'name': "router",    'host': 'newy-w2.fabric-testbed.net', 'nic': 'NIC_ConnectX_5', 'cores': 32, 'ram': 96, 'disk': 40, 'image': 'default_ubuntu_22', 'packages': ['iperf3', 'net-tools', 'moreutils','python3']}
]
net_conf = [
 {"name": "net0", "subnet": "10.0.0.0/24", "nodes": [{"name": "sender",   "addr": "10.0.0.100", 'nic': 'NIC_ConnectX_5'}, {"name": "router", "addr": "10.0.0.1", 'nic': 'NIC_ConnectX_5'}]},
 {"name": "net1", "subnet": "10.0.1.0/24", "nodes": [{"name": "receiver", "addr": "10.0.1.100", 'nic': 'NIC_Basic'}, {"name": "router", "addr": "10.0.1.1", 'nic': 'NIC_ConnectX_5'}]},
]
route_conf = [
 {"addr": "10.0.1.0/24", "gw": "10.0.0.1", "nodes": ["sender"]}, 
 {"addr": "10.0.0.0/24", "gw": "10.0.1.1", "nodes": ["receiver"]}
]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

In [ ]:
site_name="NEWY" # change it if you want to reserve your nodes in another site.

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'],
                    host=n['host'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model=node['nic'], name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Extend your slice

If you don’t plan to finish an experiment in one day, you can extend your slice. The following cell extends your reservation for 7 days.

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)

In [ ]:
# this part is for cpu pinning and numa tuning.

for n in net_conf:
    for node in n['nodes']:
        slice.get_node(node["name"]).pin_cpu(component_name=n['name'])

for node in slice.get_nodes():
    # Pin memmory for VM to same Numa node as the components
    try:
        node.numa_tune()
        
    except Exception as e:
        print(f"Warning: could not pin memory for {node.get_name()}: {e}")
        
    node.os_reboot()

### Install L4S Kernel

In [ ]:
for node in slice.get_nodes():
    # Download and unzip the kernel package
    node.execute("wget https://github.com/L4STeam/linux/releases/download/testing-build/l4s-testing.zip")
    node.execute("sudo apt install unzip")
    node.execute("unzip l4s-testing.zip")
    
    # Install the kernel packages and update GRUB
    node.execute("sudo dpkg --install debian_build/*")
    node.execute("sudo update-grub")
    node.execute("sudo reboot")

# wait for all nodes to come back up
slice.wait_ssh(progress=True)
for node in slice.get_nodes():
    # check kernel version
    node.execute("hostname; uname -a")

In [ ]:
#configuration for DUALPI2 bottleneck on the router
cmd_dualpi2="""sudo apt-get update
sudo apt -y install git gcc make bison flex libdb-dev libelf-dev pkg-config libbpf-dev libmnl-dev libcap-dev libatm1-dev selinux-utils libselinux1-dev
sudo git clone https://github.com/L4STeam/iproute2.git && cd iproute2
sudo ./configure
sudo make
sudo make install"""
slice.get_node(name="router").execute(cmd_dualpi2)
slice.get_node(name="router").execute("sudo modprobe sch_dualpi2")

### Install BBR2 Kernel

In [ ]:
pkg_list = ['linux-headers-5.13.12_5.13.12-2_amd64.deb',
            'linux-libc-dev_5.13.12-2_amd64.deb',
            'linux-image-5.13.12_5.13.12-2_amd64.deb']

cmd_BBRv2 = """sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 5.13.12"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub 
sudo reboot"""


for pkg in pkg_list:
        slice.get_node(name="sender").execute("wget https://raw.githubusercontent.com/sdatta97/imcbbrrepro/main/setup/{packet}".format(packet=pkg))
        slice.get_node(name="receiver").execute("wget https://raw.githubusercontent.com/sdatta97/imcbbrrepro/main/setup/{packet}".format(packet=pkg))

slice.get_node(name="sender").execute("sudo dpkg -i  *.deb")
slice.get_node(name="sender").execute(cmd_BBRv2)

slice.get_node(name="receiver").execute("sudo dpkg -i  *.deb")
slice.get_node(name="receiver").execute(cmd_BBRv2)

### Install BBR3 Kernel

In [ ]:
pkg_list = ['linux-headers-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-image-6.4.0+_6.4.0-g6e321d1c986a-5_amd64.deb',
            'linux-libc-dev_6.4.0-g6e321d1c986a-5_amd64.deb']

cmd_BBRv3 ="""sudo grub-set-default "Advanced options for Ubuntu>Ubuntu, with Linux 6.4.0"
sudo grub-mkconfig -o /boot/grub/grub.cfg
sudo sed -i 's/^GRUB_DEFAULT=.*/GRUB_DEFAULT=saved/' /etc/default/grub
sudo update-grub
sudo reboot"""

for pkg in pkg_list:
    slice.get_node(name="sender").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))
    slice.get_node(name="receiver").execute("wget https://github.com/ashutoshs25/bbrv3-kernel/raw/main/{packet}".format(packet=pkg))
    
slice.get_node(name="sender").execute("sudo dpkg -i  *.deb")
slice.get_node(name="sender").execute(cmd_BBRv3)

slice.get_node(name="receiver").execute("sudo dpkg -i  *.deb")
slice.get_node(name="receiver").execute(cmd_BBRv3)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo DEBIAN_FRONTEND=noninteractive apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        
        if_name1 = n['name'] + '-' + net['name'] + '-p1'
        if_name2 = n['name'] + '-' + net['name'] + '-p2'
        if_name_list=[if_name1, if_name2]
        #if_name_list=[if_name1]
        
        for if_name in if_name_list:            

            try:
                iface = slice.get_interface(if_name)
            except Exception:   
                print(f"{if_name} not found, skipping")  
                continue
            
            iface.ip_link_up()
            if n['addr']:
                iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
            else:
                iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
for net in net_conf:
    for n in net['nodes']:
        print(n)

In [ ]:
# make sure all interfaces are brought up
for iface in slice.get_interfaces():
    iface.ip_link_up()

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# turn off segmentation offload on interfaces
for iface in slice.get_interfaces():
    iface_name = iface.get_device_name()
    n = iface.get_node()
    offloads = ["gro", "lro", "gso", "tso"]
    for offload in offloads:
        n.execute("sudo ethtool -K %s %s off" % (iface_name, offload))

### Validate base network

Before we run any experiment, we should check the “base” network - before adding any emulated delay or rate limiting - and make sure that it will not be a limiting factor in the experiment.

In [ ]:
# check base delay
_ = slice.get_node("sender").execute("ping -c 5 receiver")

In [ ]:
# check base capacity (by sending 10 parallel flows, look at their sum throughput)
import time
_ = slice.get_node("receiver").execute("iperf3 -s -1 -D")
time.sleep(5)
_ = slice.get_node("sender").execute("iperf3 -t 30 -i 10 -P 3 -c receiver")

### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
filtered = [
    entry
    for entry in ifaces
    if entry.get("ip_addr") not in (None, "None")
]

In [ ]:
ifaces_new=filtered

In [ ]:
clean_edges = [
    (u, v, attr)
    for (u, v, attr) in edges
    if (u != 'None' and v != 'None')
]

In [ ]:
edges_new=clean_edges

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges_new)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Tune Hosts

In [ ]:
for node in slice.get_nodes():
    node.upload_directory('/home/fabric/work/Internship-work/node_tools','.')
    node.execute('chmod +x node_tools/host_tune.sh')
    node.execute('sudo sysctl net.ipv4.tcp_available_congestion_control')
    node.execute('sudo ./node_tools/host_tune.sh')

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
# For DPDK
router_node.upload_directory('/home/fabric/work/DPDK/node_toolsv2','.')
router_node.execute('chmod +x node_toolsv2/install.sh')
router_node.execute('sudo ./node_toolsv2/install.sh')

In [ ]:
# For DPDK
stdout, stderr = router_node.execute(f"sudo node_toolsv2/grub.sh {router_node.get_ram()}", quiet=True, output_file=f"logs/{router_node.get_name()}-grub.log")
stdout, stderr = router_node.execute("sudo node_toolsv2/apply_vfio_settings.sh", quiet=True, output_file=f"logs/{router_node.get_name()}-vfio.log")

In [ ]:
# Install Mellanox Drivers (I uploaded it to my google drive, but you can also check https://network.nvidia.com/products/infiniband-drivers/linux/mlnx_ofed/)
router_node.execute("pip install gdown")
router_node.execute("~/.local/bin/gdown https://drive.google.com/uc?id=1Cy0gDm-44sKhaEXyEeuKQSx2di7DO1Xd")

In [ ]:
router_node.execute("sudo mount -o ro,loop /home/ubuntu/MLNX_OFED_LINUX-24.10-3.2.5.0-ubuntu22.04-x86_64.iso /mnt/")
router_node.execute("sudo /mnt/mlnxofedinstall --force") 
router_node.execute("sudo /etc/init.d/openibd restart")

In [ ]:
router_node.execute("sudo reboot")
# wait for all nodes to come back up
slice.wait_ssh(progress=True)

In [ ]:
# necessary for DPDK
cmds = """
sudo apt-get update
sudo apt-get install -y libelf-dev libssl-dev libbsd-dev libarchive-dev libfdt-dev libjansson-dev
"""
router_node.execute(cmds)

### Experiments

In [ ]:
router_node = slice.get_node("router")
sender_node = slice.get_node("sender")
receiver_node = slice.get_node("receiver")

In [ ]:
router_ingress_iface = router_node.get_interface(network_name = "net0")
router_ingress_name = router_ingress_iface.get_device_name()
router_egress_iface  = router_node.get_interface(network_name = "net1")
router_egress_name = router_egress_iface.get_device_name()

In [ ]:
sender_egress_iface = sender_node.get_interface(network_name = "net0")
sender_egress_name = sender_egress_iface.get_device_name()

receiver_ingress_iface  = receiver_node.get_interface(network_name = "net1")
receiver_ingress_name = receiver_ingress_iface.get_device_name()

In [ ]:
sender_node.execute("uname -r")
router_node.execute("uname -r")
receiver_node.execute("uname -r")

In [ ]:
# generate full factorial experiment
import itertools

exp_factors = {
    'n_bdp': [8],  # n x bandwidth delay product (BDP)
    'btl_capacity': [8], #in Gbps 
    'base_rtt': [100], # in ms 
    'aqm': ['single-queue-FQ'],
    'ecn_threshold': [100], # in ms
    'ecn': [1],  # 0: noecn, 1: ecn, 3: accecn #'rx_L4S_ecn': [0, 1, 3]
    'cc': ["cubic"], # prague, bbr2, cubic, bbr
    'flow_number': [1], # flow per file
    'duration': [60],
    'ssthresh_bdp_level':[0.9],
    'reset_timer':[0], # in terms of RTT
    'file_size':[400], # in GB
    'trial': [1,2,3,4,5,6,7,8,9,10] # it means total 10 trials here.
}
factor_names = [k for k in exp_factors]
factor_lists = list(itertools.product(*exp_factors.values()))

exp_lists = []

seen_combinations = set()

# Removing ECN factor from FIFO bottleneck because it does not support ECN. This could be done also for pie_drop and codel_drop but we have not done.
# Removing the cases where ECN Threshold is less than or equal to the buffer size in time, these cases are not meaningful in practice.

for factor_l in factor_lists:
    temp_dict = dict(zip(factor_names, factor_l))
    
    if temp_dict['n_bdp'] * temp_dict['base_rtt'] >= temp_dict['ecn_threshold']:

        if temp_dict['aqm'] == 'FIFO':
            del temp_dict['ecn_threshold']
        # Convert dict to a frozenset for set operations
        fs = frozenset(temp_dict.items())
    
        if fs not in seen_combinations:
            seen_combinations.add(fs)
            exp_lists.append(temp_dict)

data_dir = slice_name + '-same-site'

print("Number of experiments:",len(exp_lists))

In [ ]:
sender_node.execute('sudo sysctl -w net.core.wmem_default=$((64*1024*1024))')
sender_node.execute('sudo sysctl -w net.core.rmem_default=$((64*1024*1024))')

sender_node.execute('sudo sysctl -w net.core.wmem_max=$((1024*1024*1024))')
sender_node.execute('sudo sysctl -w net.core.rmem_max=$((1024*1024*1024))')

sender_node.execute('sudo sysctl -w net.ipv4.tcp_wmem="$((6*1024*1024)) $((64*1024*1024)) $((1024*1024*1024))"')
sender_node.execute('sudo sysctl -w net.ipv4.tcp_rmem="$((6*1024*1024)) $((64*1024*1024)) $((1024*1024*1024))"')

In [ ]:
receiver_node.execute('sudo sysctl -w net.core.wmem_default=$((64*1024*1024))')
receiver_node.execute('sudo sysctl -w net.core.rmem_default=$((64*1024*1024))')

receiver_node.execute('sudo sysctl -w net.core.wmem_max=$((1024*1024*1024))')
receiver_node.execute('sudo sysctl -w net.core.rmem_max=$((1024*1024*1024))')

receiver_node.execute('sudo sysctl -w net.ipv4.tcp_wmem="$((6*1024*1024)) $((64*1024*1024)) $((1024*1024*1024))"')
receiver_node.execute('sudo sysctl -w net.ipv4.tcp_rmem="$((6*1024*1024)) $((64*1024*1024)) $((1024*1024*1024))"')

In [ ]:
# check the buffer sizes
sender_node.execute("sysctl net.core.rmem_max net.core.wmem_max net.ipv4.tcp_rmem net.ipv4.tcp_wmem")
receiver_node.execute("sysctl net.core.rmem_max net.core.wmem_max net.ipv4.tcp_rmem net.ipv4.tcp_wmem")
router_node.execute("sysctl net.core.rmem_max net.core.wmem_max net.ipv4.tcp_rmem net.ipv4.tcp_wmem")

In [ ]:
import time


# make sure that you are using correct kernel. 


# make sure BBR is available # this is BBRv3 in BBRv3 kernel, otherwise it is BBRv1. 
# In BBRv3 kernel, BBRv1 is bbr1 not bbr.


#sender_node.execute("sudo modprobe tcp_bbr") 
#receiver_node.execute("sudo modprobe tcp_bbr")

# if you use BBRv2 kernel, you can do this.

#sender_node.execute("sudo modprobe tcp_bbr2")
#receiver_node.execute("sudo modprobe tcp_bbr2")


old_flow_number=0

for exp in exp_lists:

    if exp['aqm']=="FIFO":
        exp['ecn_threshold']=0


    if exp['cc']=="prague":
        exp['ecn']=3
        
    name = str(exp['cc'])+"_" +str(exp['aqm']) +"_" +str(exp['ecn']) +"_" +str(exp['ecn_threshold']) +"_" +str(exp['n_bdp'])+"_"+str(exp['btl_capacity'])+"_"+str(exp['base_rtt'])+"_"+ str(exp['flow_number']) +"_" +str(exp['duration'])+"_"+str(exp['file_size']) +"_"+str(exp['ssthresh_bdp_level'])+"_"+str(exp['reset_timer'])+"_"+str(exp['trial'])
    name_tx= name+ ".txt"
    
    
    stdout_tx_json, stderr_tx_json = receiver_node.execute("ls " + name_tx, quiet=True) 
    

    if len(stdout_tx_json):
        print("Already have " + name_tx + ", skipping")

    elif len(stderr_tx_json):

        print("Running:",exp)

        if exp['cc'] =="bbr3":
            
            exp['cc'] = "bbr"

        sender_node.execute("sudo modprobe tcp_" + exp['cc'])

        receiver_node.execute("sudo modprobe tcp_" + exp['cc'])
        receiver_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc'])

        receiver_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['ecn'])) 
        
        # fixed values
        btl_limit    = int(1000*exp['n_bdp']*exp['btl_capacity']*1000*exp['base_rtt']/8) # limit of the bottleneck, n_bdp x BDP in bytes 
        packet_number=int(btl_limit/1500)+1

        # set router buffer limit 
        bdp_kbyte = exp['base_rtt']*exp['btl_capacity']*1000/8

        btl_limit_ss=int(1000*1000*exp['btl_capacity']*(exp['base_rtt'])/8)

        
        btl_limit_ss=(btl_limit_ss/1500)

        
        ssthreshinit=btl_limit_ss # BDP initialization

        
        sender_node.execute("sudo sysctl -w net.ipv4.tcp_congestion_control=" + exp['cc'])
        sender_node.execute("sudo sysctl -w net.ipv4.tcp_ecn="+str(exp['ecn'])) 


        receiver_node.execute(f"sudo tc qdisc del dev {receiver_ingress_name} root")
        receiver_node.execute("sudo tc qdisc replace dev " + receiver_ingress_name + " root netem delay " + str(exp['base_rtt']) + "ms limit 10000000")


        # clean up
        receiver_node.execute("sudo killall iperf3")
        sender_node.execute("sudo killall iperf3")
        
        time.sleep(5) 

        #router_node.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.4/dpdk-stable-22.11.4/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm01_1flow.cfg 2 > output-{name}.log 2>&1")   

        router_node.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.4/dpdk-stable-22.11.4/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10.sh {int(exp['btl_capacity']*1000)} user_01_tm01_1flow-{exp['ecn_threshold']}ECN.cfg 2 > output-{name}.log 2>&1")   


        time.sleep(1) 

        router_node.execute_thread(f"sudo bash /home/ubuntu/DPDK/v22.11.4/dpdk-stable-22.11.4/DaaS/PoCPhase3/tm10/tm10_scripts/run_tm10-v2.sh {int(exp['btl_capacity']*1000)} user_01_tm-rev.cfg 2 > output-reverse-{name}.log 2>&1")

        time.sleep(1)
        
        # start an iperf3 receiver
        receiver_node.execute_thread(f"iperf3 -s -1 -i 0.1 -p 33000 -fm --logfile " + name + ".txt")

    
        time.sleep(10) 

        
        #sender_node.execute_thread("sleep 1; iperf3 -c receiver -p 33000 --cport 1025 -fm -n " + str(exp['file_size']) + "G"+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) +" -J > "+ name+".json")

        #sender_node.execute("sleep 1; iperf3 -c receiver -p 33000 --cport 30001 -fm -n " + str(exp['file_size']) + "G"+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) +" -J > "+ name+".json")

        #sender_node.execute_thread("sleep 1; iperf3 -c receiver -p 33000 --cport 1025 -fm -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) +" -J > "+ name+".json")

        sender_node.execute_thread("sleep 1; iperf3 -Z -c receiver -p 33000 --cport 30001 -t " + str(exp['duration'])+ " -C " + exp["cc"] + " -P "+str(exp['flow_number']) +" -J > "+ name+".json")

        
        #time.sleep(exp['duration'] + 10)

        
        #router_node.execute("sudo pkill -f tm10")

        #router_node.execute("sudo rm -f /mnt/huge/rte*map_*")

        #receiver_node.execute("sudo killall iperf3")
        #sender_node.execute("sudo killall iperf3")

        #time.sleep(25) 

        time.sleep(exp['duration'] + 10)

        router_node.execute("sudo pkill -f tm10")

        router_node.execute("sudo rm -f /mnt/huge/rte*map_*")

        receiver_node.execute("sudo killall iperf3")
        sender_node.execute("sudo killall iperf3")

        time.sleep(25) 
      
print("done")

In [ ]:
cmds_py_install = '''
            sudo apt -y install python3-pip
            pip install numpy
            pip install matplotlib
            pip install pandas
            '''

sender_node.execute(cmds_py_install)
receiver_node.execute(cmds_py_install)

In [ ]:
data_dir="8GBPS-DPDK-SQ-prague-bbr"

sender_node.execute('mkdir '+data_dir)
sender_node.execute('mv *.json '+ data_dir)
sender_node.execute('mv *.txt '+ data_dir)

receiver_node.execute('mkdir '+data_dir)
receiver_node.execute('mv *.txt '+ data_dir)

In [ ]:
data_dir="cubic-dpdk-results"

In [ ]:
sender_node.upload_file("/home/fabric/work/Internship-work/analysis_with_ss.py", f"/home/ubuntu/{data_dir}/analysis.py")

In [ ]:
receiver_node.upload_file("/home/fabric/work/Internship-work/analysis_receiver.py", f"/home/ubuntu/{data_dir}/analysis.py")

In [ ]:
sender_node.execute(f'python3 /home/ubuntu/{data_dir}/analysis.py')

In [ ]:
receiver_node.execute(f'python3 /home/ubuntu/{data_dir}/analysis.py')

In [ ]:
sender_node.download_file("/home/fabric/work/DPDK/new-paper-exps/8GB-fct-cubic-SQ-DPDK.json",f"/home/ubuntu/{data_dir}/fct_data.json")
sender_node.download_file("/home/fabric/work/DPDK/new-paper-exps/8GB-retrans-cubic-SQ-DPDK.json",f"/home/ubuntu/{data_dir}/retransmits_data.json")
sender_node.download_file("/home/fabric/work/DPDK/new-paper-exps/8GB-rtt-cubic-SQ-DPDK.json",f"/home/ubuntu/{data_dir}/stream_rtt_data.json")

In [ ]:
receiver_node.download_file("/home/fabric/work/DPDK/new-paper-exps/8GB-cubic-SQ-DPDK-receiver_stats.json",f"/home/ubuntu/{data_dir}/final_stream_stats.json")
receiver_node.download_file("/home/fabric/work/DPDK/new-paper-exps/8GB-cubic-SQ-DPDK-receiver_stats-jfi.json",f"/home/ubuntu/{data_dir}/fairness_index.json")

### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()